In [1]:
#####
pmetric = 'overall_weighted_f1'
cancer = 'BRCA'
file = '../../data/exact_match/best_models_BRCA.tsv'
file_preds = '../../src/feature_list_with_performance_with_subtype_names_20200828.tsv.gz'
file_fts = '../../src/collected_features_matrix_20200722.tsv.gz'
file_tarball = '../../src/tarball/{}_v8_20200203.tsv'.format(cancer)
out = '../../data/heatmap/BRCA_matrix_best.tsv'
#####

In [2]:
import pandas as pd

In [3]:
master_preds = pd.read_csv(file_preds, sep = '\t')
# master_preds

/Users/leejor/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Find best model across all groups
master_fts = pd.read_csv(file, sep = '\t', index_col = 0)

best = {'team':0, 'performance':0}
for c in master_fts.columns:
    # Fix naming for cloudforest
    if c.startswith('CF|'):
        c = c.replace(' ','_')
    subset = master_preds[master_preds['featureID']==c]
    subset = subset[subset['cohort'] == cancer]
    m = subset[subset['performance_metric']== pmetric].sort_values(by='Mean', ascending=False).reset_index(drop=True)['Mean']
    m = m[0]
    if m > best['performance']:
        best['team'] = c
        best['performance'] = m
    elif m == best['performance']:
        print('equal performance found, need to update code')
print('best model across all groups: ', best)
    
# Pull features of that model
df = pd.read_csv('../../data/exact_match/best_models_BRCA.tsv', sep = '\t', index_col=0)
fts = df.index[df[best['team']] == 1].tolist()
print(len(fts), 'fts in best model')     

best model across all groups:  {'team': 'AKLIMATE_BRCA_reduced_model_1000_feature_set_BRCA', 'performance': 0.9259999999999999}
1000 fts in best model


In [5]:
# Get values of original matrix for features of interest
tarball = pd.read_csv(file_tarball, sep = '\t', index_col = 0)

In [6]:
# tarball

In [7]:
mat = tarball[['Labels']+ fts]
mat.to_csv(out, sep='\t')